In [1]:
import os
import pygame
import numpy as np

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class AmazonBoard(object):
    
    def __init__(self):
        self.N = 8
        self.board = np.zeros((self.N, self.N), dtype=np.int)	# 0 for blank, 3 for block
        self.player1pos = [[0, 2], [2, 0], [5, 0], [7, 2]]
        self.player2pos = [[0, 5], [2, 7], [5, 7], [7, 5]]
        self.validsteps = []
        for pos in self.player1pos:
            self.board[pos[0]][pos[1]] = 1
        for pos in self.player2pos:
            self.board[pos[0]][pos[1]] = 2
        self.player = 1
        self.showboard = np.zeros((self.N, self.N), dtype=np.object)
        self.history = []

    def __getitem__(self, key):
        return self.board[key]
        
    def _changeplayer(self):
        self.player = 3 - self.player
        
    def inboard(self, x, y):
        return min(x, y) >= 0 and max(x, y) < self.N
        
    def _gensteps(self):
        self.validsteps = []
        startposes = self.player1pos if self.player == 1 else self.player2pos
        dx = [1, 1, 1, 0, 0, -1, -1, -1]
        dy = [-1, 0, 1, -1, 1, -1, 0, 1]
        for pos in startposes:
            for dir in range(8):
                for dis in range(1, self.N):
                    x = pos[0] + dx[dir] * dis
                    y = pos[1] + dy[dir] * dis
                    if not self.inboard(x, y):
                        break
                    if self.board[x][y] == 0:
                        for dir2 in range(8):
                            for dis2 in range(1, self.N):
                                x2 = x + dx[dir2] * dis2
                                y2 = y + dy[dir2] * dis2
                                if max(x2, y2) >= self.N or min(x2, y2) < 0:
                                    break
                                elif self.board[x2][y2] == 0:
                                    self.validsteps.append(pos + [x, y, x2, y2])
                                    continue
                                else:
                                    break
                    else:
                        break
        
    def draw(self):
        dic = { 0: ' ', 1: 'X', 2: 'O', 3: '*' }
        for i in range(self.N):
            for j in range(self.N):
                self.showboard[i][j] = dic[self.board[i][j]]
        print(self.showboard)
        
    def step(self, trial):    # python list
        if max(trial) > self.N or min(trial) < 0:
            print("Step out of board.")
            return False
        self._gensteps()
        if trial not in self.validsteps:
            print("Invalid step.")
            return False
        self.board[trial[0]][trial[1]] = 0
        self.board[trial[2]][trial[3]] = self.player
        self.board[trial[4]][trial[5]] = 3
        poses = self.player1pos if self.player == 1 else self.player2pos
        poses.remove([trial[0], trial[1]])
        poses.append([trial[2], trial[3]])
        self.history.append(trial)
        self._changeplayer()
    
    def endgame(self):
        self.validsteps = []
        self._gensteps()
        return len(self.validsteps) == 0

In [6]:
class AmazonUI(object):
    
    def __init__(self):
        pygame.init()
        self.board = AmazonBoard()
        self.N = self.board.N
        self.width = 720
        self.height = 720
        
        self.status = 0  # 0 for idle, 1 for selected
        self.movefrom = []
        self.moveto = []
        
        self.gridwidth = (self.width // (self.N + 2)) 
        self.screen = pygame.display.set_mode((self.width, self.height))
        self.basex = self.gridwidth
        self.basey = 1 * self.gridwidth
        pygame.display.set_caption("Amazon by Lzzz")
        self.FPS = 80
        self.clock = pygame.time.Clock()
        self.resource_folder = os.path.dirname('.')
        self.background_img = pygame.image.load(os.path.join(self.resource_folder, 'images/background.png')).convert()
        self.background_img = pygame.transform.scale(self.background_img, (720, 720))
        blank_img = pygame.image.load(os.path.join(self.resource_folder, 'images/blank.png'))
        black_img = pygame.image.load(os.path.join(self.resource_folder, 'images/black.png'))
        white_img = pygame.image.load(os.path.join(self.resource_folder, 'images/white.png'))
        block_img = pygame.image.load(os.path.join(self.resource_folder, 'images/block.png'))
        yellow_img = pygame.image.load(os.path.join(self.resource_folder, 'images/yellow.png'))
        red_img = pygame.image.load(os.path.join(self.resource_folder, 'images/red.png'))
        new_img = pygame.image.load(os.path.join(self.resource_folder, 'images/new.png'))
        self.imgs = [blank_img, black_img, white_img, block_img, yellow_img, red_img, new_img]
        for i in range(len(self.imgs)):
            self.imgs[i] = pygame.transform.scale(self.imgs[i], (self.gridwidth, self.gridwidth))
        self.running = False
        self.WHITE = (255, 255, 255)
        self.BLACK = (0, 0, 0)
        self.GREEN = (0, 255, 0)
    
    def getvalid(self):
        dx = [1, 1, 1, 0, 0, -1, -1, -1]
        dy = [-1, 0, 1, -1, 1, -1, 0, 1]
        if self.status == 0:
            return self.board.player1pos if self.board.player == 1 else self.board.player2pos
        else:
            x0, y0 = self.movefrom if self.status == 1 else self.moveto
            ret = []
            for dir in range(8):
                for dis in range(1, self.N):
                    x = x0 + dx[dir] * dis
                    y = y0 + dy[dir] * dis
                    if not self.board.inboard(x, y) or self.board[x][y] != 0:
                        break
                    ret.append([x, y])
            return ret
    
    def run(self):
        self.running = True
        while self.running:
            self.clock.tick(self.FPS)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    x, y = event.pos
                    x = int((x - self.basex) / self.gridwidth)
                    y = int((y - self.basey) / self.gridwidth)
                    if not self.board.inboard(x, y):
                        continue
                    if self.board[x][y] == 3 or self.board[x][y] == 3 - self.board.player:  # block
                        continue
                    if self.board[x][y] == self.board.player:
                        self.status = 1
                        self.movefrom = [x, y]
                        continue
                    # board[x][y] == 0
                    if [x, y] not in self.getvalid():
                        continue
                    if self.status == 0:
                        self.movefrom = [x, y]
                        self.status = 1
                    elif self.status == 1:
                        self.moveto = [x, y]
                        self.status = 2
                    else:
                        self.board.step(self.movefrom + self.moveto + [x, y])
                        self.status = 0
            self.draw_background(self.screen)
            if self.board.endgame():
                print("end")
            pygame.display.flip()
            
    def draw_background(self, surf):
        surf.blit(self.background_img, (0, 0))
        for i in range(self.N):
            for j in range(self.N):
                surf.blit(self.imgs[self.board[i][j]], (self.gridwidth * i + self.basex, self.gridwidth * j + self.basey))
        if self.status > 0:
            pros = self.getvalid()
            img = self.imgs[4] if self.status == 1 else self.imgs[5]
            for x, y in pros:
                surf.blit(img, (self.gridwidth * x + self.basex, self.gridwidth * y + self.basey))
            if self.status == 2:
                x, y = self.moveto
                surf.blit(self.imgs[6], (self.gridwidth * x + self.basex, self.gridwidth * y + self.basey))
        

In [7]:
game = AmazonUI()

In [ ]:
game.run()